# VRA + Beamforming in the mode frequency domain

Contact: https://github.com/gherold (Gert Herold)

- Example simulation of a stationary and a rotating source ("b11c", see also https://doi.org/10.14279/depositonce-8460)
- VRA + frequency domain beamforming
- separation of rotating source / stationary source

Requirements
- Acoular "rotating2024" branch (https://github.com/gherold/acoular/tree/rotating2024, last tested: 2025-03-24)
- Array geometry file "ring64circ.xml"

## 1) Initial setup

Imports:

In [ ]:
import acoular as ac
ac.config.global_caching='none'
ac.config.h5library = "h5py"
#ac.config.h5library = "tables"
import pylab as plt
c0 = 343.


array geometry:

In [ ]:
m = ac.MicGeomCirc(file = './ring64circ.xml')
channels = m.ringlist[0].mics

## 2) Simulation

Skip this and continue at section 3) if you already simulated the data.

Sound source positions and other parameters:

In [ ]:
z = 0.5 # m
r = 0.25 # m
phi = 90./180*plt.pi # source @ 12 o'clock)

rpm = -1500 # rpm
rps = rpm/60.

In [ ]:
tmax = 10 # s

sfreq = 48000 # Hz
nsamples = tmax * sfreq

n1 = ac.WNoiseGenerator(sample_freq=sfreq, num_samples=nsamples, seed = 1, rms=1.0)
n2 = ac.WNoiseGenerator(sample_freq=sfreq, num_samples=nsamples, seed = 2, rms=0.5**0.5)

stationary source

In [ ]:
p1 = ac.PointSource(signal = n1, 
                    mics = m, 
                    loc = ( r * plt.cos(phi), 
                            r * plt.sin(phi), 
                            z))

rotating source

In [ ]:
tr = ac.Trajectory()
delta_t = 1./plt.abs(rps)/16.0 # 16 steps per revolution
for t in plt.arange(0, tmax*1.001, delta_t):
    phit = t * rps * 2 * plt.pi #angle
    tr.points[t] = (r * plt.cos(phi + phit), 
                    r * plt.sin(phi + phit), 
                    z)

p2 = ac.MovingPointSource(signal=n2, mics=m, trajectory=tr)

In [ ]:
p = ac.SourceMixer(sources = [p1, p2])

wh5 = ac.WriteH5(source=p, file = 'b11c_notrigger.h5')
wh5.save()

## 3) Evaluation

focus grid

In [ ]:
g = ac.RectGrid(x_min = -0.3,
                x_max =  0.3, 
                y_min = -0.3,
                y_max =  0.3,
                z = 0.5,
                increment = 0.01)

print(f'number grid points: {g.size}')

plot setup

In [ ]:
plt.figure(1,(7,7))
plt.plot(0,0,'+')

for i in plt.arange(m.num_mics):
    plt.plot(m.pos[0,i],m.pos[1,i],'oC0')
    plt.text(m.pos[0,i]+0.005,m.pos[1,i]+0.005,str(i+1), fontsize=6)
    
plt.plot(g.pos[0],g.pos[1], '+k', ms=3)
plt.title(m.file)
plt.axis('equal')
plt.show()

time signal:

In [ ]:
timedata = ac.TimeSamples(file = 'b11c_notrigger.h5')
print(f'signal duration: {timedata.num_samples/timedata.sample_freq} s')

Setup virtual rotation

In [ ]:
frot = rps

env = ac.AxialRotatingFlowEnvironment(c=c0)

print(frot)

Virtual rotation (or not)

In [ ]:
# space domain -> mode domain
pmt0 = ac.SpaceModesTransformer(source = timedata, channel_order = channels)

# VRA
pmtr = ac.VirtualRotatorModal(source = pmt0, 
                              rotational_speed = rps, 
                              delay = plt.abs(g.z)/c0)


Setup beamformer:

In [ ]:

# Sound propagation model
sts = ac.SteeringVector(grid=g, 
                        mics=m, 
                        env=env)
# steering vector also into mode domain
st = ac.SteeringVectorModeTransformer(steer = sts,  
                                      channel_order = channels)

# CSM
f = ac.PowerSpectra(window = 'Hanning', 
                      overlap = '50%',
                      block_size = 256)

# Beamformer
bf = ac.BeamformerBase(freq_data = f,
                        steer= st)


# classic beamforming in spatial domain
"""
f2 = ac.PowerSpectra(source = timedata,
                     window = 'Hanning', 
                      overlap = '50%',
                      block_size = 256)
bf2 = ac.BeamformerBase(freq_data = f2,
                        steer= sts,
                        r_diag = True)
"""


Calculation and visualization

In [ ]:
# For getting rid of unwanted "rotational noise", set r_diag to True

bf.r_diag = True
#bf.r_diag = False

#freqbands
freqs = [2500, 5000, 10000] 
#bandsize
band = 3

nfreqs = len(freqs)

fig = plt.figure(3,figsize=(nfreqs*4, 8))  
nsub = 0


for pmt in {pmt0, pmtr}:
    f.source = pmt
    if hasattr(pmt,'rotational_speed'):
        env.rotational_speed = -frot
        addstr = f'frot={pmt.rotational_speed:.2f}Hz'
    else:
        env.rotational_speed = 0
        addstr = 'stationary'

    for freq in freqs:
        nsub+=1
    
        bbmap = bf.synthetic(freq,band)
        Lp = ac.L_p(bbmap)
        mx = Lp.max()
        
    
        plt.subplot(2,nfreqs, nsub) 
    
        #bf result
        plt.imshow(Lp.T,
                   vmin = mx-20,
                   vmax = mx,
                   origin='lower',
                   extent=g.extend(),
                   interpolation='nearest',
                   cmap=plt.cm.hot_r)
    

        plt.title(f'{freq} Hz, {addstr}')
        plt.colorbar()


plt.show()